In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import os
import numpy as np
import linecache #fast access to a specific file line
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from einops import rearrange, repeat
import torch.nn.functional as F
import time
import torchinfo
from pathlib import Path

In [2]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

1.11.0
11.1
True


In [3]:
torch.multiprocessing.set_sharing_strategy('file_system') #to avoid issues in the dataloading
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
CONT_SIZE = 30

In [5]:
ALPHABET = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I",
            "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]

ALPHABET = {ALPHABET[i]:i for i in range(len(ALPHABET))}

ALPHABET['-']= 20
ALPHABET['Z']= 21

print(ALPHABET)

{'A': 0, 'R': 1, 'N': 2, 'D': 3, 'C': 4, 'Q': 5, 'E': 6, 'G': 7, 'H': 8, 'I': 9, 'L': 10, 'K': 11, 'M': 12, 'F': 13, 'P': 14, 'S': 15, 'T': 16, 'W': 17, 'Y': 18, 'V': 19, '-': 20, 'Z': 21}


In [6]:
#max 4
rep = torch.tensor([4, 4, 2, 1, 1, 1, 1, 1, 1, 1, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4])
rand = torch.tensor([0, 0, 0.2, 0, 0, 0, 0, 0, 0, 0.5, 0.7, 0, 0, 0, 0, 0, 0, 0,0, 0])
#max 8 
rep = torch.tensor([8, 8, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 5, 7, 8, 8, 8, 8, 8])
rand = torch.tensor([0, 0, 0.2, 0, 0, 0, 0, 0, 0.2, 0.5, 0.3, 0.9, 0.8, 0.5, 0.9, 0, 0, 0, 0, 0])

In [7]:
class MyDataset(Dataset):
    def __init__(self, data_dir, cont_size=6,div=2000,verbose=False):
        """
        Initialize the dataset by precomputing a bunch of data on the sequence families
        """
        self.col_size = 60 #number of column per file (Fasta standard)
        self.data_dir = data_dir #directory of the dataset
        self.cont_size = cont_size
        self.div = div
        self.len = 0  #number of families of sequences (1 per file)
        self.paths = {} #path of each families in the folder
        self.seq_lens = {} #length of each member of the family
        self.seq_nums = {} #number of member of the family
        self.aa_freqs = {} #frequencies of each symbol in the sequence family
        self.p_aa_freqs = {} #frequencies of each symbol in each sequence of a family
        
        
        dir_path = data_dir
        count = 0
        
        # Iterate directory
        for path in os.listdir(dir_path):
            # check if current path is a file
            temp_path = os.path.join(dir_path, path)
            if os.path.isfile(temp_path):
                n = 0 #number of sequences
                p = 0 # used to calculate the length of the sequences
                r = 0 # also used this way

                l = 0 # length of the seq l = p * self.col_size + r 

                cpt = 0 # to detect inconsistencies
                
                with open(temp_path, newline='') as f:
                    first_prot = True
                    newf = True
                    
                    aa_freq = torch.zeros(20)
                    p_aa_freq = torch.zeros(0)
                    
                    #parsing the file
                    line = f.readline()[:-1]
                    while line:
                        cpt += 1
                        if line[0] == '>': #header line
                            if not first_prot:
                                p_aa_freq = torch.cat([p_aa_freq,prot_aa_freq])
                            prot_aa_freq = torch.zeros(1,20)
                            n += 1
                            if newf and not first_prot:
                                newf = False
                            first_prot = False
                                
                        else:# sequence line
                            if newf and len(line) == self.col_size:
                                p += 1

                            if newf and len(line) != self.col_size:
                                r = len(line)
                            for aa in line:
                                aa_id = ALPHABET.get(aa,21)
                                if aa_id < 20:
                                    aa_freq[aa_id] += 1
                                    prot_aa_freq[0][aa_id] += 1

                            assert len(line) == self.col_size or len(line) == r
                        line = f.readline()[:-1]
                    
                    p_aa_freq = torch.cat([p_aa_freq,prot_aa_freq])
                    aa_freq = F.normalize(aa_freq,dim=0,p=1)
                    p_aa_freq = F.normalize(p_aa_freq,dim=1,p=1)

                l = p*self.col_size + r
                
                #sanity check
                #if the file line count is coherent with the number of sequences and their line count
                try: #if r != 0
                    assert (p+2) * n == cpt
                except: #if r == 0
                    assert (p+1) * n == cpt
                    assert r == 0
                    
                
                if n>1: #if this is false, we can't find pairs
                    self.paths[count] = path
                    self.seq_lens[count] = l
                    self.seq_nums[count] = n
                    self.aa_freqs[count] = aa_freq
                    self.p_aa_freqs[count] = p_aa_freq
                    count += 1
                    
                    if verbose and (count % 100 ==0) : print(f"seen = {count}")
            
        self.len = count
    
    def __len__(self):
        return self.len
     
    def sample(self, high, low=0, s=1):
        sample = np.random.choice(high-low, s, replace=False)
        return sample + low
    
    def __getitem__(self, idx, sample_size='auto',rep=rep,rand=rand): 
        """
        input idx of the family of the sample
        return a Tensor containing several samples from the family corresponding to the index
        """
        
        X = []
        y = []
        
        PIDs = []
        local_PIDs = []
        
        pfreqs = []
        local_pfreqs = []
        
        lengths = []
        
        pos = []
        
        bin_n = len(rep) #for biasing the sampling
        
        precomputed_pos = [] #positions of the amino-acids
        for i in range(-self.cont_size,self.cont_size+1):
            precomputed_pos.append(i)
        for i in range(-self.cont_size,0):
            precomputed_pos.append(i)
        for i in range(1,self.cont_size+1):
            precomputed_pos.append(i)
        
        precomputed_pos = torch.tensor(precomputed_pos).float()
        
        data_path = os.path.join(self.data_dir, self.paths[idx])
        try:
            n = self.seq_nums[idx]
            l = self.seq_lens[idx]
        except:
            print(idx)
            pass
        
        #sampling more for big families and long sequences
        if type(sample_size) != int:
            sample_s = min(n * l,25_000)
            coef = round((sample_s)/self.div) 
            sample_size = max(1,coef)
        
        p = l // self.col_size
        r = l % self.col_size # l = p * q + r
        sequence_line_count = p+2 if r else p+1

        for _ in range(sample_size):
            i,j = self.sample(n,s=2)

            start_i = 2 + (sequence_line_count)*i #start line of protein i
            start_j = 2 + (sequence_line_count)*j #start line of protein j
            
            seq_i = ''
            seq_j = ''
            
            PID_ij = 0
            
            l_ij = 0
            for offset in range(sequence_line_count-1): #computing PID and removing aligned '-' ##might need to compute the actual column num
                line_i = linecache.getline(data_path, (start_i + offset))[:-1]
                line_j = linecache.getline(data_path, (start_j + offset))[:-1]
                for aa_i, aa_j in zip(line_i,line_j):
                    if aa_i == aa_j:
                        if aa_i != '-':
                            PID_ij += 1
                            seq_i += aa_i
                            seq_j += aa_j        
                    else:
                        seq_i += aa_i
                        seq_j += aa_j
                    
                    if aa_i != '-' and aa_j != '-':
                        l_ij += 1
            
            try:
                PID_ij = PID_ij/l_ij
            except:
                PID_ij = 0 #case 0/0
            
            align_l = len(seq_i)
            possible_k = [] #possible position to take
            for k,(a_i,a_j) in enumerate(zip(seq_i,seq_j)):   
                if ALPHABET.get(a_i,21) < 20 and ALPHABET.get(a_j,21) < 20:
                    possible_k.append(k)
            
            # biasing for more diverse PID  
            bin_idx = int(PID_ij//(1/bin_n))
            rep_number = rep[bin_idx].clone()
            if torch.rand(1) < rand[bin_idx]:
                rep_number+=1
            
            for _ in range(rep_number):
                try:   
                    k = np.random.choice(possible_k)
                except:
                    continue
                
                #adding to the output
                lengths.append(align_l)
                pos_ij = (k + precomputed_pos)
                pos.append(pos_ij)
                
                #computing the windows
                window_i = ''
                window_j = ''
                for w in range(k-self.cont_size,k+self.cont_size+1):
                    if w < 0 or w >= align_l: #case of the edges
                        window_i += 'Z'
                        window_j += 'Z'
                    else:
                        window_i += seq_i[w]
                        window_j += seq_j[w]

                y_j = ALPHABET.get(window_j[self.cont_size], 21) # 'Z' is the default value for rare AA
                X_i = [ALPHABET.get(i, 21) for i in (window_i+window_j[:self.cont_size]+window_j[self.cont_size+1:])]       

                X.append(X_i)
                y.append(y_j)
                PIDs.append(PID_ij)
                #computing the local PID
                local_PID_ij = sum(1 for AA1,AA2 in zip(window_i, window_j[:self.cont_size]) if AA1 == AA2 and ALPHABET.get(AA1,21) < 20) \
                             + sum(1 for AA1,AA2 in zip(reversed(window_i), reversed(window_j[self.cont_size+1:])) if AA1 == AA2 and ALPHABET.get(AA1,21) < 20)

                loc_comp = sum(1 for AA1,AA2 in zip(window_i, window_j[:self.cont_size]) if ALPHABET.get(AA1,21) < 20 and ALPHABET.get(AA2,21) < 20) \
                             + sum(1 for AA1,AA2 in zip(reversed(window_i), reversed(window_j[self.cont_size+1:])) if ALPHABET.get(AA1,21) < 20 and ALPHABET.get(AA2,21) < 20)
                try:
                    tmp = local_PID_ij/loc_comp  
                except:
                    tmp = 0 #case 0/0

                local_PIDs.append(tmp)
                pfreqs.append(self.aa_freqs[idx])
                p_i_freqs = self.p_aa_freqs[idx][i]
                p_j_freqs = self.p_aa_freqs[idx][j]

                local_pfreqs.append(torch.stack((p_i_freqs,p_j_freqs)))

                assert y_j < 20
                assert X_i[self.cont_size] < 20
            
        linecache.clearcache() #clearing the cache
        X = torch.tensor(X)
        try:
            X = F.one_hot(X,22)[:,:,0:-1]
        except RuntimeError:
            pass
        if len(pos) == 0:
            pos = torch.tensor(pos)
        else:
            pos = torch.stack(pos)
        pfreqs = torch.stack(pfreqs)
        local_pfreqs = torch.stack(local_pfreqs)
        X = X.float()
        y = torch.tensor(y)
        PIDs = torch.tensor(PIDs)
        local_PIDs = torch.tensor(local_PIDs)
        lengths = torch.tensor(lengths)
        out = X,y.long(),PIDs,local_PIDs,pfreqs,local_pfreqs,pos,lengths
        return out


In [8]:
#Run This for new data
"""
train_dataset = MyDataset(r"data/train_data",cont_size = 6,4000)
test_dataset = MyDataset(r"data/test_data",cont_size = 6,div=2000)
val_dataset = MyDataset(r"data/val_data",cont_size = 6,div=2000)

fname = 'data/train_dataset1.pth'
savepath = Path(fname)
with savepath.open("wb") as fp:
    torch.save(train_dataset,fp)
    
fname = 'data/test_dataset1.pth'
savepath = Path(fname)
with savepath.open("wb") as fp:
    torch.save(test_dataset,fp)
    
fname = 'data/val_dataset1.pth'
savepath = Path(fname)
with savepath.open("wb") as fp:
    torch.save(val_dataset,fp)
"""

'\ntrain_dataset = MyDataset(r"data/train_data",cont_size = 6,4000)\ntest_dataset = MyDataset(r"data/test_data",cont_size = 6,div=2000)\nval_dataset = MyDataset(r"data/val_data",cont_size = 6,div=2000)\n\nfname = \'data/train_dataset1.pth\'\nsavepath = Path(fname)\nwith savepath.open("wb") as fp:\n    torch.save(train_dataset,fp)\n    \nfname = \'data/test_dataset1.pth\'\nsavepath = Path(fname)\nwith savepath.open("wb") as fp:\n    torch.save(test_dataset,fp)\n    \nfname = \'data/val_dataset1.pth\'\nsavepath = Path(fname)\nwith savepath.open("wb") as fp:\n    torch.save(val_dataset,fp)\n'

In [9]:
#To load datasets with all features computed
fname = 'data/train_dataset1.pth'
savepath = Path(fname)
with savepath.open("rb") as fp:
    train_dataset = torch.load(fp)
    
fname = 'data/test_dataset1.pth'
savepath = Path(fname)
with savepath.open("rb") as fp:
    test_dataset = torch.load(fp)
    
fname = 'data/val_dataset1.pth'
savepath = Path(fname)
with savepath.open("rb") as fp:
    val_dataset = torch.load(fp)
    
print(len(train_dataset))
print(len(test_dataset))
print(len(val_dataset))

train_dataset.cont_size = CONT_SIZE
test_dataset.cont_size = CONT_SIZE
val_dataset.cont_size = CONT_SIZE

train_dataset.div = 2000
test_dataset.div = 2000
val_dataset.div = 2000

13219
2838
2826


In [10]:
def my_collate(batch):
    """
    Transforms a list of tensors to a batch tensor
    """
    data = torch.cat([item[0] for item in batch],dim=0)
    target = torch.cat([item[1] for item in batch],dim=0)
    PID = torch.cat([item[2] for item in batch],dim=0)
    lPID = torch.cat([item[3] for item in batch],dim=0)
    pfreqs = torch.cat([item[4] for item in batch],dim=0)
    lpfreqs = torch.cat([item[5] for item in batch],dim=0)
    pos = torch.cat([item[6] for item in batch],dim=0)
    length = torch.cat([item[7] for item in batch],dim=0)
    return data, target, PID, lPID,pfreqs,lpfreqs, pos, length

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True,collate_fn=my_collate,num_workers=4)

test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True,collate_fn=my_collate,num_workers=4)

val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True,collate_fn=my_collate,num_workers=4)

In [12]:
class AttBlock(nn.Module):
    """
    Self-Attention Block of the Transformer
    """
    def __init__(self,in_features,out_features=None,num_heads=8,head_dims=24):
        super().__init__()
        out_features = out_features or in_features
        
        self.Q_w = nn.Linear(in_features,num_heads*head_dims,bias=False)
        self.K_w = nn.Linear(in_features,num_heads*head_dims,bias=False)
        self.V_w = nn.Linear(in_features,num_heads*head_dims,bias=False)
        
        self.att = nn.MultiheadAttention(num_heads*head_dims,num_heads=num_heads,batch_first=True)
        self.lin = nn.Linear(num_heads*head_dims,out_features)
        
    def forward(self,x):
        Q = self.Q_w(x)
        K = self.K_w(x)
        V = self.V_w(x)
        out,_ = self.att(Q,K,V,need_weights=False)
        out = self.lin(out)
        
        return out

In [13]:
class CrossAttBlock(nn.Module):
    """
    Cross-Attention Block of the Transformer
    """
    def __init__(self,in_features,out_features=None,num_heads=8,head_dims=24):
        super().__init__()
        out_features = out_features or in_features
        
        self.Q_w = nn.Linear(in_features,num_heads*head_dims,bias=False)
        self.K_w = nn.Linear(in_features,num_heads*head_dims,bias=False)
        self.V_w = nn.Linear(in_features,num_heads*head_dims,bias=False)
        
        self.att = nn.MultiheadAttention(num_heads*head_dims,num_heads=num_heads,batch_first=True)
        self.lin = nn.Linear(num_heads*head_dims,out_features)
        
    def forward(self,x):
        src,tgt = x[:,0],x[:,1]
        Q = self.Q_w(tgt)
        K = self.K_w(src)
        V = self.V_w(src)
        out,_ = self.att(Q,K,V,need_weights=False)
        out = self.lin(out)
        
        return out

In [14]:
class RowAttBlock(nn.Module):
    """
    Row-wise Attention Block
    """
    def __init__(self,in_features,out_features=None,num_heads=8,head_dims=24,num_seq=2):
        super().__init__()
        self.att_block = AttBlock(in_features,num_heads=num_heads,head_dims=head_dims)
        self.num_seq = num_seq
    
    def forward(self,x):
        out = rearrange(x,"b n l d -> (b n) l d",n=self.num_seq)
        out = self.att_block(out)
        out = rearrange(out,"(b n) l d -> b n l d",n=self.num_seq)
        return out
    
class ColAttBlock(nn.Module):
    """
    Column-wise Attention Block
    """
    def __init__(self,in_features,out_features=None,num_heads=8,head_dims=24,seq_len=2*CONT_SIZE+4):
        super().__init__()
        self.att_block = AttBlock(in_features,num_heads=num_heads,head_dims=head_dims)
        self.seq_len = seq_len
    
    def forward(self,x):
        out = rearrange(x,"b n l d -> (b l) n d",l=self.seq_len)
        out = self.att_block(out)
        out = rearrange(out,"(b l) n d -> b n l d",l=self.seq_len)
        return out

In [15]:
class FeedFoward(nn.Module):
    """
    MLP Block of the Transformer
    """    
    def __init__(self,in_features,out_features=None,wide_factor=4):
        super().__init__()
        out_features = out_features or in_features
        hidden_dim = wide_factor * in_features
        
        self.lin1 = nn.Linear(in_features,hidden_dim)
        self.act1 = nn.GELU()
        self.lin2 = nn.Linear(hidden_dim,out_features)
        self.act2 = nn.GELU()
        
    def forward(self,x):
        out = self.lin1(x)
        out = self.act1(out)
        out = self.lin2(out)
        out = self.act2(out)
        
        return out

In [16]:
#(from the timm library)
def drop_path(x, drop_prob: float = 0.1, training: bool = False, scale_by_keep: bool = True):
    """Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).

    This is the same as the DropConnect impl I created for EfficientNet, etc networks, however,
    the original name is misleading as 'Drop Connect' is a different form of dropout in a separate paper...
    See discussion: https://github.com/tensorflow/tpu/issues/494#issuecomment-532968956 ... I've opted for
    changing the layer and argument names to 'drop path' rather than mix DropConnect as a layer name and use
    'survival rate' as the argument.

    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = x.new_empty(shape).bernoulli_(keep_prob)
    if keep_prob > 0.0 and scale_by_keep:
        random_tensor.div_(keep_prob)
    return x * random_tensor


class DropPath(nn.Module):
    """Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """
    def __init__(self, drop_prob=None, scale_by_keep=True):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob
        self.scale_by_keep = scale_by_keep

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training, self.scale_by_keep)


In [17]:
class Block(nn.Module):
    """
    Basic Block of the Transformer
    """
    def __init__(self,in_features,num_heads=8,head_dims=24,wide_factor=4,drop=0.1,num_seq=2,seq_len=2*CONT_SIZE+4):
        super().__init__()
        
        self.row_att_block = RowAttBlock(in_features,num_heads=num_heads,head_dims=head_dims,num_seq=num_seq)
        self.col_att_block = ColAttBlock(in_features,num_heads=num_heads,head_dims=head_dims,seq_len=seq_len)
        self.ff = FeedFoward(in_features,wide_factor=wide_factor)
        self.drop_path = DropPath(drop)
        
        self.norm1 = nn.LayerNorm(in_features)
        self.norm2 = nn.LayerNorm(in_features)
        self.norm3 = nn.LayerNorm(in_features)
    def forward(self,x):
        out = x + self.drop_path(self.row_att_block(x))
        out = self.norm1(out)
        out = x + self.drop_path(self.col_att_block(out))
        out = self.norm2(out)
        out = out + self.drop_path(self.ff(out))
        out = self.norm3(out)
        return out

In [18]:
class LastBlock(nn.Module):
    """
    Last Block of the Transformer
    """
    def __init__(self,in_features,num_heads=8,head_dims=24,wide_factor=4,drop=0.1):
        super().__init__()
        
        self.cross_att_block = CrossAttBlock(in_features,num_heads=num_heads,head_dims=head_dims)
        self.ff = FeedFoward(in_features,wide_factor=wide_factor)
        self.drop_path = DropPath(drop)
        
        self.norm1 = nn.LayerNorm(in_features)
        self.norm2 = nn.LayerNorm(in_features)
    def forward(self,x):
        out = x[:,1] + self.drop_path(self.cross_att_block(x))
        out = self.norm1(out)
        out = out + self.drop_path(self.ff(out))
        out = self.norm2(out)
        return out

In [19]:
class Classifier_Head(nn.Module):
    """
    Classifier Head of the Transformer
    """
    def __init__(self,in_features,clf_dims,out_size,seq_len):
        super().__init__()
        in_dim = seq_len*in_features
        self.in_dim = in_dim
        
        layers = []
        for out_dim in clf_dims:
            layers.append(nn.Linear(in_dim,out_dim))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(p=0.2))
            in_dim = out_dim

        layers.append(nn.Linear(in_dim,out_size))
        
        self.clf = nn.Sequential(*layers)
        
    def forward(self,x):
        out = x.reshape((-1,self.in_dim))
        
        out = self.clf(out)
        
        return out

In [20]:
def get_params(input_size,N,head,head_dim,wide_factor,drop_prob):
    """
    Returns the initialization parameters of the Transformer
    """
    return input_size, [head for _ in range(N)], [head_dim for _ in range(N)], [wide_factor for _ in range(N)], [drop_prob for _ in range(N)], 

In [21]:
class AttNet(nn.Module):
    """
    Transformer-like neural net
    """
    def __init__(self,in_features,num_heads,head_dims,wide_factors,drops,input_dim=21,out_size=20,num_seq=2,seq_len=2*CONT_SIZE+4,clf_dims=[256,64],cont_size=CONT_SIZE):
        super().__init__()
        self.in_features = in_features
        self.input_dim = input_dim
        self.cont_size=cont_size
        
        blocks = []
        r = min(len(num_heads),len(head_dims),len(wide_factors),len(drops))
        for i,(n_h, h_d,w,d) in enumerate(zip(num_heads,head_dims,wide_factors,drops)):
            if i < (r-1):
                blocks.append(Block(in_features,num_heads=n_h,head_dims=h_d,wide_factor=w,drop=d,seq_len=seq_len,num_seq=num_seq))
            else:
                blocks.append(LastBlock(in_features,num_heads=n_h,head_dims=h_d,wide_factor=w,drop=d))
        self.feature_extractor = nn.Sequential(*blocks)
        
        self.clf = Classifier_Head(in_features,clf_dims,out_size=out_size,seq_len=seq_len)
        
        sp = Path("data/freq.pth")
        with sp.open("rb") as fp:
            self.F = nn.Parameter(torch.log(torch.load(fp)))
            
        pid_layers = [nn.Linear(1,2*in_features),nn.Sigmoid()]
        self.pid_l = nn.Sequential(*pid_layers)
        
    def to_input(self,x,PID,pfreqs,lpfreqs,pos,length):
        X_idx = torch.argmax(x[:,self.cont_size],dim=1)
        seq1 = x[:,:2*self.cont_size+1]
        y_freq = F.pad(F.softmax(self.F[X_idx],dim=1).unsqueeze(1), pad=(0, 1), mode='constant', value=0) 
        seq2 = torch.cat((x[:,2*self.cont_size+1:3*self.cont_size+1],y_freq,x[:,3*self.cont_size+1:]),dim=1)
        aa_pos = pos[:,:2*self.cont_size+1]/length.unsqueeze(1)
        aa_pos = aa_pos.unsqueeze(2)
        pos_dim = (self.in_features-self.input_dim-1)//2
        
        for i in range(pos_dim): #positionnal_encoding
            p = torch.cos(pos[:,:2*self.cont_size+1]/(4**(2*i/pos_dim))).unsqueeze(2)
            ip = torch.sin(pos[:,:2*self.cont_size+1]/(4**(2*i/pos_dim))).unsqueeze(2)
            aa_pos = torch.cat([aa_pos,p,ip],dim=2)
        
        seq1 = torch.cat([seq1,aa_pos],dim=2)
        seq2 = torch.cat([seq2,aa_pos],dim=2)
        
        out = torch.stack([seq1,seq2],dim=1)
        
        pad = (0,self.in_features-self.input_dim+1)
        pf = F.pad(pfreqs.unsqueeze(1),pad =pad, mode='constant', value=0)
        pf = repeat(pf,"b 1 d -> b 2 1 d")
        
        pid = self.pid_l(PID.unsqueeze(1)).unsqueeze(1)
        pid = rearrange(pid,"b 1 (n e) -> b n 1 e",n=2)
        
        lpf = F.pad(lpfreqs,pad=pad, mode='constant', value=0).unsqueeze(2)
        out = torch.cat([out,pf,pid,lpf],dim=2)
        
        return out
        
        
    def forward(self,x,PID,pfreqs,lpfreqs,pos,length):
        X_input = self.to_input(x,PID,pfreqs,lpfreqs,pos,length)
        features = self.feature_extractor(X_input)
        out = self.clf(features)
        return out

In [22]:
class State:
    """
    Used for checkpointing
    """
    def __init__(self,model,optim,scheduler):
        self.model = model
        self.optim = optim
        self.scheduler = scheduler
        self.epoch = 0

In [23]:
params = get_params(32,3,8,32,4,0.1)
model = AttNet(*params,clf_dims=[512,128,32])
torchinfo.summary(model,[(1,4*CONT_SIZE+1,21),(1,),(1,20),(1,2,20),(1,4*CONT_SIZE+1),(1,)])

Layer (type:depth-idx)                             Output Shape              Param #
AttNet                                             [1, 20]                   400
├─Sequential: 1-1                                  [1, 64]                   --
│    └─Linear: 2-1                                 [1, 64]                   128
│    └─Sigmoid: 2-2                                [1, 64]                   --
├─Sequential: 1-2                                  [1, 64, 32]               --
│    └─Block: 2-3                                  [1, 2, 64, 32]            --
│    │    └─RowAttBlock: 3-1                       [1, 2, 64, 32]            295,968
│    │    └─DropPath: 3-2                          [1, 2, 64, 32]            --
│    │    └─LayerNorm: 3-3                         [1, 2, 64, 32]            64
│    │    └─ColAttBlock: 3-4                       [1, 2, 64, 32]            295,968
│    │    └─DropPath: 3-5                          [1, 2, 64, 32]            --
│    │    └─LayerNorm: 

In [24]:
def eval_model(fname,train,test,val,N=10):
    """
    Evaluate the MSE (Brier Score) of the model
    """
    savepath = Path(fname)
    with savepath.open("rb") as fp:
        state = torch.load(fp)
        
    state.model.eval()
    with torch.no_grad():
        print('EVALUATING ON TRAIN DATA : ')
        eval_losses = []
        for _ in range(N):
            for X,y, PID, lPID,pfreqs,lpfreqs,pos,length in train:
                X = X.to(device)
                y = y.to(device)
                PID = PID.to(device)
                pos = pos.to(device)
                pfreqs = pfreqs.to(device)
                lpfreqs = lpfreqs.to(device)
                length = length.to(device)

                y_hat = state.model(X,PID,pfreqs,lpfreqs,pos,length)
                y_hat = F.softmax(y_hat,dim=1)

                y = F.one_hot(y, 20)
                eval_l = (y_hat-y)**2
                eval_losses.append(torch.sum(eval_l,dim=1).detach().cpu())

        score_train = torch.mean(torch.cat(eval_losses)).detach().cpu().item()
        print(f"{score_train = }\n")

        print('EVALUATING ON TEST DATA : ')
        eval_losses = []
        for _ in range(N):
            for X,y, PID, lPID,pfreqs,lpfreqs, pos,length in test:
                X = X.to(device)
                y = y.to(device)
                PID = PID.to(device)
                pos = pos.to(device)
                pfreqs = pfreqs.to(device)
                lpfreqs = lpfreqs.to(device)
                length = length.to(device)

                y_hat = state.model(X,PID,pfreqs,lpfreqs,pos,length)
                y_hat = F.softmax(y_hat,dim=1)

                y = F.one_hot(y, 20)
                eval_l = (y_hat-y)**2
                eval_losses.append(torch.sum(eval_l,dim=1).detach().cpu())
    
        score_test = torch.mean(torch.cat(eval_losses)).detach().cpu().item()
        print(f"{score_test = }\n")

        print('EVALUATING ON VAL DATA : ')
        eval_losses = []
        for _ in range(N):
            for X,y, PID, lPID,pfreqs,lpfreqs, pos, length in val:
                X = X.to(device)
                y = y.to(device)
                PID = PID.to(device)
                lPID = lPID.to(device)
                pos = pos.to(device)
                pfreqs = pfreqs.to(device)
                lpfreqs = lpfreqs.to(device)
                length = length.to(device)

                y_hat = state.model(X,PID,pfreqs,lpfreqs,pos,length)
                y_hat = F.softmax(y_hat,dim=1)

                y = F.one_hot(y, 20)
                eval_l = (y_hat-y)**2
                eval_losses.append(torch.sum(eval_l,dim=1).detach().cpu())
    
        score_val = torch.mean(torch.cat(eval_losses)).detach().cpu().item()
        print(f"{score_val = }\n")
    
    return score_train, score_test, score_val

In [25]:
def eval_model_PID(fname,train,test,val,N=10):
    """
    Evaluate the MSE (Brier Score) of the model for different slices of PID in the data
    """
    savepath = Path(fname)
    with savepath.open("rb") as fp:
        state = torch.load(fp)
    bin_n = 20
    state.model.eval()
    with torch.no_grad():
        print('EVALUATING ON TRAIN DATA : ')
        eval_losses = []
        PIDS = []
        for _ in range(N):
            for X,y, PID, lPID,pfreqs,lpfreqs,pos,length in train:
                PIDS.append(PID)
  
                X = X.to(device)
                y = y.to(device)
                PID = PID.to(device)
                pos = pos.to(device)
                pfreqs = pfreqs.to(device)
                lpfreqs = lpfreqs.to(device)
                length = length.to(device)

                y_hat = state.model(X,PID,pfreqs,lpfreqs,pos,length)
                y_hat = F.softmax(y_hat,dim=1)

                y = F.one_hot(y, 20)
                eval_l = (y_hat-y)**2
                eval_losses.append(torch.sum(eval_l,dim=1).detach().cpu())
        PIDS = torch.cat(PIDS)
        BINS = (PIDS/(1/bin_n)).long()
        scores = torch.cat(eval_losses).detach().cpu()
        
        Losses_train = torch.zeros(bin_n)
        bar_train = torch.zeros(bin_n)
        for i,n in zip(BINS,scores):
            bar_train[i] += 1
            Losses_train[i] += n
        Losses_train = Losses_train/bar_train
        plt.bar(np.arange(bin_n)/bin_n+(1/(2*bin_n)),Losses_train,width=1/(bin_n+1))
        plt.title("Brier Score wrt PID on Train dataset")
        plt.show()
        plt.bar(np.arange(bin_n)/bin_n+(1/(2*bin_n)),bar_train,width=1/(bin_n+1))
        plt.title("Example count wrt PID on Train dataset")
        plt.show()
        
        print('EVALUATING ON TEST DATA : ')
        eval_losses = []
        PIDS = []
        for _ in range(N):
            for X,y, PID, lPID,pfreqs,lpfreqs,pos,length in test:
                PIDS.append(PID)
  
                X = X.to(device)
                y = y.to(device)
                PID = PID.to(device)
                pos = pos.to(device)
                pfreqs = pfreqs.to(device)
                lpfreqs = lpfreqs.to(device)
                length = length.to(device)

                y_hat = state.model(X,PID,pfreqs,lpfreqs,pos,length)
                y_hat = F.softmax(y_hat,dim=1)

                y = F.one_hot(y, 20)
                eval_l = (y_hat-y)**2
                eval_losses.append(torch.sum(eval_l,dim=1).detach().cpu())
        PIDS = torch.cat(PIDS)
        BINS = (PIDS/(1/bin_n)).long()
        scores = torch.cat(eval_losses).detach().cpu()
        
        Losses_test = torch.zeros(bin_n)
        bar_test = torch.zeros(bin_n)
        for i,n in zip(BINS,scores):
            bar_test[i] += 1
            Losses_test[i] += n
        Losses_test = Losses_test/bar_test
        plt.bar(np.arange(bin_n)/bin_n+(1/(2*bin_n)),Losses_test,width=1/(bin_n+1))
        plt.title("Brier Score wrt PID on Test dataset")
        plt.show()
        plt.bar(np.arange(bin_n)/bin_n+(1/(2*bin_n)),bar_test,width=1/(bin_n+1))
        plt.title("Example count wrt PID on Test dataset")
        plt.show()
        
        print('EVALUATING ON VAL DATA : ')
        eval_losses = []
        PIDS = []
        for _ in range(N):
            for X,y, PID, lPID,pfreqs,lpfreqs,pos,length in val:
                PIDS.append(PID)
  
                X = X.to(device)
                y = y.to(device)
                PID = PID.to(device)
                pos = pos.to(device)
                pfreqs = pfreqs.to(device)
                lpfreqs = lpfreqs.to(device)
                length = length.to(device)

                y_hat = state.model(X,PID,pfreqs,lpfreqs,pos,length)
                y_hat = F.softmax(y_hat,dim=1)

                y = F.one_hot(y, 20)
                eval_l = (y_hat-y)**2
                eval_losses.append(torch.sum(eval_l,dim=1).detach().cpu())
        PIDS = torch.cat(PIDS)
        BINS = (PIDS/(1/bin_n)).long()
        scores = torch.cat(eval_losses).detach().cpu()
        
        Losses_val = torch.zeros(bin_n)
        bar_val = torch.zeros(bin_n)
        for i,n in zip(BINS,scores):
            bar_val[i] += 1
            Losses_val[i] += n
        Losses_val = Losses_val/bar_val
        plt.bar(np.arange(bin_n)/bin_n+(1/(2*bin_n)),Losses_val,width=1/(bin_n+1))
        plt.title("Brier Score wrt PID on Val dataset")
        plt.show()
        plt.bar(np.arange(bin_n)/bin_n+(1/(2*bin_n)),bar_val,width=1/(bin_n+1))
        plt.title("Example count wrt PID on Val dataset")
        plt.show()
        
    return Losses_train, bar_train, Losses_test, bar_test, Losses_val,bar_val

In [26]:
def eval_model_acc_PID(fname,train,test,val,N=10):
    """
    Evaluate the accuracy of the model for different slices of PID in the data
    """
    savepath = Path(fname)
    with savepath.open("rb") as fp:
        state = torch.load(fp)
    bin_n = 20
    state.model.eval()
    with torch.no_grad():
        print('EVALUATING ON TRAIN DATA : ')
        eval_losses = []
        PIDS = []
        for _ in range(N):
            for X,y, PID, lPID,pfreqs,lpfreqs,pos,length in train:
                PIDS.append(PID)
  
                X = X.to(device)
                y = y.to(device)
                PID = PID.to(device)
                pos = pos.to(device)
                pfreqs = pfreqs.to(device)
                lpfreqs = lpfreqs.to(device)
                length = length.to(device)

                y_hat = state.model(X,PID,pfreqs,lpfreqs,pos,length)
                y_hat = torch.argmax(y_hat,dim=1)

                eval_l = (y == y_hat).long()
                eval_losses.append(eval_l.detach().cpu())
                
        PIDS = torch.cat(PIDS)
        BINS = (PIDS/(1/bin_n)).long()
        scores = torch.cat(eval_losses).detach().cpu()
        
        Losses_train = torch.zeros(bin_n)
        bar_train = torch.zeros(bin_n)
        for i,n in zip(BINS,scores):
            bar_train[i] += 1
            Losses_train[i] += n
        Losses_train = Losses_train/bar_train
        plt.bar(np.arange(bin_n)/bin_n+(1/(2*bin_n)),Losses_train,width=1/(bin_n+1))
        plt.title("Accuracy wrt PID on Train dataset")
        plt.show()
        plt.bar(np.arange(bin_n)/bin_n+(1/(2*bin_n)),bar_train,width=1/(bin_n+1))
        plt.title("Example count wrt PID on Train dataset")
        plt.show()
        
        print('EVALUATING ON TEST DATA : ')
        eval_losses = []
        PIDS = []
        for _ in range(N):
            for X,y, PID, lPID,pfreqs,lpfreqs,pos,length in test:
                PIDS.append(PID)
  
                X = X.to(device)
                y = y.to(device)
                PID = PID.to(device)
                pos = pos.to(device)
                pfreqs = pfreqs.to(device)
                lpfreqs = lpfreqs.to(device)
                length = length.to(device)

                y_hat = state.model(X,PID,pfreqs,lpfreqs,pos,length)
                y_hat = torch.argmax(y_hat,dim=1)

                eval_l = (y == y_hat).long()
                eval_losses.append(eval_l.detach().cpu())
        PIDS = torch.cat(PIDS)
        BINS = (PIDS/(1/bin_n)).long()
        scores = torch.cat(eval_losses).detach().cpu()
        
        Losses_test = torch.zeros(bin_n)
        bar_test = torch.zeros(bin_n)
        for i,n in zip(BINS,scores):
            bar_test[i] += 1
            Losses_test[i] += n
        Losses_test = Losses_test/bar_test
        plt.bar(np.arange(bin_n)/bin_n+(1/(2*bin_n)),Losses_test,width=1/(bin_n+1))
        plt.title("Accuracy wrt PID on Test dataset")
        plt.show()
        plt.bar(np.arange(bin_n)/bin_n+(1/(2*bin_n)),bar_test,width=1/(bin_n+1))
        plt.title("Example count wrt PID on Test dataset")
        plt.show()
        
        print('EVALUATING ON VAL DATA : ')
        eval_losses = []
        PIDS = []
        for _ in range(N):
            for X,y, PID, lPID,pfreqs,lpfreqs,pos,length in val:
                PIDS.append(PID)
  
                X = X.to(device)
                y = y.to(device)
                PID = PID.to(device)
                pos = pos.to(device)
                pfreqs = pfreqs.to(device)
                lpfreqs = lpfreqs.to(device)
                length = length.to(device)

                y_hat = state.model(X,PID,pfreqs,lpfreqs,pos,length)
                y_hat = torch.argmax(y_hat,dim=1)

                eval_l = (y == y_hat).long()
                eval_losses.append(eval_l.detach().cpu())
        PIDS = torch.cat(PIDS)
        BINS = (PIDS/(1/bin_n)).long()
        scores = torch.cat(eval_losses).detach().cpu()
        
        Losses_val = torch.zeros(bin_n)
        bar_val = torch.zeros(bin_n)
        for i,n in zip(BINS,scores):
            bar_val[i] += 1
            Losses_val[i] += n
        Losses_val = Losses_val/bar_val
        plt.bar(np.arange(bin_n)/bin_n+(1/(2*bin_n)),Losses_val,width=1/(bin_n+1))
        plt.title("Accuracy wrt PID on Val dataset")
        plt.show()
        plt.bar(np.arange(bin_n)/bin_n+(1/(2*bin_n)),bar_val,width=1/(bin_n+1))
        plt.title("Example count wrt PID on Val dataset")
        plt.show()
        
    return Losses_train, bar_train, Losses_test, bar_test, Losses_val,bar_val

In [27]:
def main(train_loader,val_loader,epochs=101,fname="models/state.pth",fnameb=None,state=None,last_epoch_sched=float('inf'),use_mut=True,cont_size=CONT_SIZE):
    """
    Trains a model
    """
    #to get the best model
    best = float('inf')
    
    #getting the acceleration device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    #loading from previous checkpoint
    if fnameb is None:
        fnameb = fname[:-4] + '_best' +fname[-4:]
        
    savepath = Path(fname)
    if savepath.is_file():
        with savepath.open("rb") as fp:
            state = torch.load(fp)
    else:
        if state is None:
            model = AttNet(22,[8,8,8],[24,24,24],[4,4,4],[0.1,0.1,0.1])
            model = model.to(device)
            optim = torch.optim.AdamW(model.parameters(),lr = 0.0001)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim,16,2)
            state = State(model,optim,scheduler)
    
    
    Loss = nn.CrossEntropyLoss(reduction='sum')
    LossMut = nn.BCELoss(reduction='sum')
    EvalLoss = nn.MSELoss(reduction='mean')
    
    #for logs
    List_Loss = []
    Eval_Loss = []
    for epoch in range(state.epoch, epochs):
        batch_losses = []
        state.model.train()
        for X,y, PID, lPID,pfreqs,lpfreqs,pos,length in train_loader:
            X = X.to(device)
            y = y.to(device)
            PID = PID.to(device)
            pos = pos.to(device)
            pfreqs = pfreqs.to(device)
            lpfreqs = lpfreqs.to(device)
            length = length.to(device)
            
            state.optim.zero_grad()
            y_hat = state.model(X,PID,pfreqs,lpfreqs,pos,length)
            

            if use_mut:
                X_idx = torch.argmax(X[:,cont_size],dim=1)
                y_true = (X_idx == y).float().unsqueeze(1)  #0 if a mutation happens else 1 
                y_pred = F.softmax(y_hat,dim=1)
                y_pred = y_pred.gather(1,X_idx.view(-1,1)) #the Xth component of y_hat should be predicting ^
                l = (Loss(y_hat,y) + LossMut(y_pred,y_true))/440 
            else:
                l = Loss(y_hat,y)/440 
            l.backward()
            state.optim.step()
            
            
            
            batch_losses.append(l.detach().cpu())
        List_Loss.append(torch.mean(torch.stack(batch_losses)).detach().cpu())
        state.epoch = epoch + 1
        if epoch < last_epoch_sched:
            state.scheduler.step()
        
        savepath = Path(fname)
        with savepath.open("wb") as fp:
            torch.save(state,fp)
        
        with torch.no_grad():
            eval_losses = [] 
            state.model.eval()
            for X,y, PID, lPID,pfreqs,lpfreqs, pos,length in val_loader:

                X = X.to(device)
                y = y.to(device)
                PID = PID.to(device)
                pos = pos.to(device)
                pfreqs = pfreqs.to(device)
                lpfreqs = lpfreqs.to(device)
                length = length.to(device)

                y_hat = state.model(X,PID,pfreqs,lpfreqs,pos,length)
                y_hat = F.softmax(y_hat,dim=1)

                y = F.one_hot(y, 20)
                eval_l = (y_hat-y)**2
                eval_losses.append(torch.sum(eval_l,dim=1).detach().cpu())
    
            score = torch.mean(torch.cat(eval_losses)).detach().cpu().item()
            Eval_Loss.append(score)
        
        if score < best :
            best = score
            savepath = Path(fnameb)
            with savepath.open("wb") as fp:
                torch.save(state,fp)
        
        print(f"epoch n°{epoch} : train_loss = {List_Loss[-1]}, val_loss = {Eval_Loss[-1]}") 


        
    return List_Loss,Eval_Loss,state

In [ ]:
params = get_params(32,4,8,32,4,0.1)
model = AttNet(*params,clf_dims=[512,128,32])
model = model.to(device)
optim = torch.optim.AdamW(model.parameters(),lr = 0.0003)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optim,16,2)
state = State(model,optim,scheduler)

fname = "models/state_4L_MSA.pth" 
start = time.time()
Train,Eval,_ = main(train_dataloader, val_dataloader,fname=fname,epochs=2032,state=state,use_mut=False)
stop = time.time()
print(stop-start)

In [ ]:
plt.plot(np.arange(len(Train)),Train)

In [ ]:
plt.plot(np.arange(len(Eval)),Eval)

In [ ]:
N = 10 #moving average
smothed_Eval = np.convolve(Eval, np.ones(N)/N, mode='valid')
plt.plot(np.arange(len(smothed_Eval)),smothed_Eval)

In [ ]:
fname = "models/state_4L_MSA.pth" 
SCORES = eval_model(fname,train_dataloader,test_dataloader,val_dataloader)

In [ ]:
fname = "models/state_4L_MSA.pth" 
SCORES = eval_model_PID(fname,train_dataloader,test_dataloader,val_dataloader)

In [ ]:
fname = "models/state_4L_MSA.pth" 
SCORES = eval_model_acc_PID(fname,train_dataloader,test_dataloader,val_dataloader)